In [6]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col 
import statsmodels.api as sm

Good job with the column names... it helped me figure out the key problem.

Obviously still not the formula... it's missing 2 variables in each column. I know why now (next point)
You need to include all rows in your data for the regressions. A single regression is supposed to have a variable indicating whether or not the row is about BOW. You obviously can't do this on your mini datasets that have only BOW or only Competitors.
You didn't ABCD and print your data, but I bet your "hisp_over_med" variable wasn't only 0s and 1s.
No need to type the regressions twice and print them by themselves and then all together at the end.


In [2]:
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [14]:
# adjust this next line to drop the BMO rows
bank_tract = bank_tract.query('which_bank' != "BMO")

ValueError: expr must be a string to be evaluated, <class 'bool'> given

In [4]:
# create vars
bank_tract ['hisp_rate']    = (bank_tract ['HispanicLatinoPop']) / bank_tract ['Tot.Pop'] * 100
bank_tract['hisp_over_med'] = bank_tract["hisp_rate"] >  bank_tract["hisp_rate"].median()
bank_tract ['log_num_apps'] = np.log(bank_tract['num_applications'])

0        False
1        False
2        False
3        False
4        False
         ...  
13832     True
13833     True
13834     True
13835     True
13836     True
Name: hisp_over_med, Length: 13837, dtype: bool

In [10]:
# define the regression models (YOU'LL NEED TO MAKE ONE MORE VARIABLE ABOVE, AND THEN UPDATE THESE TO MATCH FORMULA)
m1 = sm.OLS.from_formula('denial_rate ~ hisp_rate', data=bank_tract).fit()
m2 = sm.OLS.from_formula('denial_rate ~ hisp_over_med', data=bank_tract).fit()
m3 = sm.OLS.from_formula('log_num_apps ~ hisp_rate', data=bank_tract).fit()
m4 = sm.OLS.from_formula('log_num_apps ~ hisp_over_med', data=bank_tract).fit()

In [11]:
# set up the formatting for the table
info_dict = {'No. observations': lambda x: f"{int(x.nobs):d}"}
float_format = '%0.3f'


In [12]:
# UPDATE THIS AS NEEDED: 
regressor_order = ['Intercept', 'hisp_rate', 'hisp_over_med']


In [13]:
# UPDATE THE COLUMN NAMES (just using the y variable in each column works)
table = summary_col(results=[m1, m2, m3, m4],
                    model_names=['|all banks denial rate reg|',
                                 '|BOW denial rate reg|',
                                 '|all banks log num apps reg|',
                                 '|BOW log num apps reg|'],
                    regressor_order=regressor_order,
                    float_format=float_format,
                    info_dict=info_dict,
                    stars=True)  

table.title = 'OLS Regressions'

# print the table
print(table)

                                                      OLS Regressions
                      |all banks denial rate reg| |BOW denial rate reg| |all banks log num apps reg| |BOW log num apps reg|
---------------------------------------------------------------------------------------------------------------------------
Intercept             0.834***                    0.829***              3.075***                     2.985***              
                      (0.002)                     (0.002)               (0.023)                      (0.019)               
hisp_rate             -0.000***                                         -0.005***                                          
                      (0.000)                                           (0.001)                                            
hisp_over_med[T.True]                             -0.019***                                          -0.187***             
                                                  (0.003)     